In [290]:
import random 
import copy

class Puzzle():

    # Constructor to design the goal state and initial state
    def __init__(self):
        random.seed(42)
        self.goalState = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
        self.state = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
        self.maxNode = 0

    def maxNodes(self,n):
        self.maxNode = n
        if self.maxNode > n:
            print('Error')

    # Check if it is the goal state    
    def goalCheck(self, state):
        return state == self.goalState
    
    # Set to the goal state
    def setGoalState(self):
        self.state = copy.deepcopy(self.goalState)
    
    # Get a list of legitimate options 
    def getActions(self, state):
        
        # Mark the index of the blank tile
        for rowIndex, row in enumerate(state):
            for colIndex, element in enumerate(row):
                if element == 0:
                    blankRow = rowIndex
                    blankCol = colIndex
        
        # A list to record actions
        actions = []
        
        # Define available actions based on the position of the blank tile
        if blankRow == 0:
            actions.append("down")
        elif blankRow == 1:
            actions.extend(("up", "down"))
        elif blankRow == 2:
            actions.append("up")
            
        if blankCol == 0:
            actions.append("right")
        elif blankCol == 1:
            actions.extend(("left", "right"))
        elif blankCol == 2:
            actions.append("left")
            
        # Randomly shuffle the actions to remove ordering bias
        random.shuffle(actions)

        return actions, blankRow, blankCol
    
    def setState(self, state):
        state = state.replace(" ", "")
        for i in range(len(self.state)):
            for j in range(len(self.state[i])):
                self.state[i][j] = state[j + 3*i];
      
    def randomizeState(self, n):
        # Take a random series of moves backwards from the goal state
        # Sets the current state of the puzzle to a state thta is guaranteed to be solvable

        currState = (self.goalState)
        
        # Iterate through the number of moves
        for i in range(n):
            actions, _, _ = self.getActions(currState)
            random_move = random.choice(actions)
            currState = self.move(currState, random_move)
          
        # Set the state of the puzzle to the random state  
        self.state = currState
            
        
    def move(self, state, action):
        # Move the blank in the specified direction
        # Returns the new state resulting from the move
        actions, blankRow, blankCol = self.getActions(state)

        newState = copy.deepcopy(state)

        # Check to make sure action is allowed given board state
        if action not in actions:
            print("Move not allowed\nAllowed moves:", actions)
            return False

        # Execute the move as a series of if statements, probably not the most efficient method
        else:
            if action == "down":
                temp = state[blankRow + 1][blankCol]
                newState[blankRow][blankCol] = temp
                newState[blankRow + 1][blankCol] = 0
            elif action == "up":
                temp = state[blankRow - 1][blankCol]
                newState[blankRow][blankCol] = temp
                newState[blankRow - 1][blankCol] = 0
            elif action == "right":
                temp = state[blankRow][blankCol + 1]
                newState[blankRow][blankCol] = temp
                newState[blankRow][blankCol + 1] = 0
            elif action == "left":
                temp = state[blankRow][blankCol - 1]
                newState[blankRow][blankCol] = temp
                newState[blankRow][blankCol -1] = 0
                
        return newState
              
    

    def printState(self):
        # Display the state of the board in the format "b12 345 678"
        rows = []

        # Iterate through all the tiles
        for row in self.state:
            for element in row:
                rows.append(str(element))
        
        # Print out the resulting state       
        print("".join(rows[0:3]), "".join(rows[3:6]), "".join(rows[6:9]))

                    
    def h1(self, state):
        # Calculate and return the h1 heuristic for a given state
        # The h1 heuristic is the number of tiles out of place from their goal position

        # Flatten the lists for comparison
        stateFlatten = sum(state, [])
        goalFlatten = sum(self.goalState, [])
        heuristic = 0

        # Iterate through the lists and compare elements
        for i, j in zip(stateFlatten, goalFlatten):
            if i != j:
                heuristic += 1

        
        return heuristic

    def h2(self, state):
        # Calculates h2

        stateDict = {}
        goalDict = {}
        heuristic = 0
        
        # Create dictionaries of the current state and goal state
        for rowIndex, row in enumerate(state):
            for colIndex, element in enumerate(row):
                stateDict[element] = (rowIndex, colIndex)
        
        for rowIndex, row in enumerate(self.goalState):
            for colIndex, element in enumerate(row):
                goalDict[element] = (rowIndex, colIndex)
                
        for tile, position in stateDict.items():
            # Skip the distance of the blank 
            if tile == 0:
                pass
            else:
                # Calculate the h2 heuristic
                goalPos = goalDict[tile]
                heuristic += (abs(position[0] - goalPos[0]) + abs(position[1] - goalPos[1]))

        return heuristic

    def calculateTotalCost(self, node_depth, state, heuristic):
        # Returns the total cost of a state given its depth and the heuristic: cost + heuristic(h1 or h2)

        if heuristic == "h2":
            return node_depth + self.h2(state)
        elif heuristic == "h1":
            return node_depth + self.h1(state)
        

    def A_star(self, heuristic="h2", print_solution=True):
        # Performs a-star search
        # Prints the list of solution moves and the solution length

        self.maxNodes(10000)

        # A dictionary for the frontier and for the expanded nodes
        frontier_nodes = {}
        expanded_nodes = {}
        
        self.starting_state = copy.deepcopy(self.state)
        currState = copy.deepcopy(self.state)
        # Node index is used for indexing the dictionaries and to keep track of the number of nodes expanded
        node_index = 0

        # Set the first element in both dictionaries to the starting state
        # This is the only node that will be in both dictionaries
        expanded_nodes[node_index] = {"state": currState, "parent": "root", "action": "start",
                                   "total_cost": self.calculateTotalCost(0, currState, heuristic), "depth": 0}
        
        frontier_nodes[node_index] = {"state": currState, "parent": "root", "action": "start",
                                   "total_cost": self.calculateTotalCost(0, currState, heuristic), "depth": 0}
        

        failure = False

        # A priority queue, where each element in the list is a tuple consisting of node index and total cost of the node.
        all_frontier_nodes = [(0, frontier_nodes[0]["total_cost"])]

        # Iterate until max node number is reached
        while not failure:

            # Get current depth of state to calculate total cost
            currDepth = 0
            for node_num, node in expanded_nodes.items():
                if node["state"] == currState:
                    currDepth = node["depth"]

            # get all legitimate actions of a state
            actions, _, _ = self.getActions(currState)

            # Iterate through possible actions 
            for action in actions:
                repeat = False

                # If max nodes reached, break out of loop
                if node_index >= self.maxNode:
                    failure = True
                    print("No Solution Found in first {} nodes generated".format(self.maxNode))
                    self.num_nodes_generated = self.maxNode
                    break

                # Find the new state corresponding to the action and calculate total cost
                newState = self.move(currState, action)
                newState_parent = copy.deepcopy(currState)

                # Check to see if new state has already been expanded
                for expanded_node in expanded_nodes.values():
                    if expanded_node["state"] == newState:
                        if expanded_node["parent"] == newState_parent:
                            repeat = True

                # Check to see if new state and parent is on the frontier
                # The same state can be added twice to the frontier if the parent state is different
                for frontier_node in frontier_nodes.values():
                    if frontier_node["state"] == newState:
                        if frontier_node["parent"] == newState_parent:
                            repeat = True

                # If new state has already been expanded or is on the frontier, continue with next action     
                if repeat:
                    continue

                else:
                    # Each action represents another node generated
                    node_index += 1
                    depth = currDepth + 1

                    # Total cost is path length (number of steps from starting state) + heuristic
                    newState_cost = self.calculateTotalCost(depth, newState, heuristic)

                    # Add the node index and total cost to the all_nodes list
                    all_frontier_nodes.append((node_index, newState_cost))

                    # Add the node to the frontier 
                    frontier_nodes[node_index] = {"state": newState, "parent": newState_parent, "action": action, "total_cost": newState_cost, "depth": currDepth + 1}

            # Sort all the nodes on the frontier by total cost
            all_frontier_nodes = sorted(all_frontier_nodes, key=lambda x: x[1])

            # If the number of nodes generated does not exceed max nodes, find the best node and set the current state to that state
            if not failure:
                # The best node will be at the front of the queue
                # After selecting the node for expansion, remove it from the queue
                best_node = all_frontier_nodes.pop(0)
                best_node_index = best_node[0]
                best_node_state = frontier_nodes[best_node_index]["state"]
                currState = best_node_state

                # Move the node from the frontier to the expanded nodes
                expanded_nodes[best_node_index] = (frontier_nodes.pop(best_node_index))
                
                # Check if current state is goal state
                if self.goalCheck(best_node_state):
                    # Create attributes for the expanded nodes and the frontier nodes
                    self.expanded_nodes = expanded_nodes
                    self.frontier_nodes = frontier_nodes
                    self.num_nodes_generated = node_index + 1

                    # Display the solution path
                    self.success(expanded_nodes, node_index, print_solution)
                    break 
                    
    def beam(self, k=10, print_solution=True):
        # Performs local beam search to solve the eight puzzle
        # k: number of child states after each iteration
        # f(n) = h1 + h2, at each iteration, the next set of nodes will be the k nodes with the lowest score

        self.starting_state = copy.deepcopy(self.state)
        starting_state = copy.deepcopy(self.state)
        # Check to see if the current state is already the goal
        if starting_state == self.goalState:
            self.success(node_dict={}, num_nodes_generated=0)

        # A dictionary of all states generated
        all_nodes= {}

        # Index for all nodes dictionary
        node_index = 0

        all_nodes[node_index] = {"state": starting_state, "parent": "root", "action": "start"}

        # Score for starting state
        starting_score = self.h1(starting_state) + self.h2(starting_state)

        # Available nodes is all the possible states that can be accessed from the current state stored as an (index, score) tuple
        available_nodes = [(node_index, starting_score)]
                
        failure = False
        success = False

        while not failure:

            # Check to see if the number of nodes generated exceeds max nodes
            if node_index >= self.maxNode:
                failure = True
                print("No Solution Found in first {} generated nodes".format(self.maxNode))
                break
              
            # Successor nodes are all the nodes that can be reached from all of the available states. At each iteration, this is reset to an empty list  
            successor_nodes = []

            # Iterate through all the possible nodes that can be visited
            for node in available_nodes:

                repeat = False

                # Current state
                currState = all_nodes[node[0]]["state"]

                # Get actions corresponding to the currState above
                actions, _, _ = self.getActions(currState)

                # Iterate through allowed actions
                for action in actions:
                    # Find the successor state 
                    successor_state = self.move(currState, action)

                    # Check if the state is visited
                    for node_num, node in all_nodes.items():
                        if node["state"] == successor_state:
                            if node["parent"] == currState:
                                repeat = True

                    # Check if the state is the goal state
                    # If it is, stop iteration
                    if successor_state == self.goalState:	
                        all_nodes[node_index] = {"state": successor_state, 
                                "parent": currState, "action": action}
                        self.expanded_nodes = all_nodes
                        self.num_nodes_generated = node_index + 1
                        self.success(all_nodes, node_index, print_solution)
                        success = True
                        break


                    if not repeat:
                        node_index += 1
                        # Calculate the score of the state
                        score = (self.h1(successor_state) + self.h2(successor_state))
                        # Add the state to the list of of nodes
                        all_nodes[node_index] = {"state": successor_state, "parent": currState, "action": action}
                        # Add the state to the successor_nodes list
                        successor_nodes.append((node_index, score))
                    else:
                        continue

                    
            # The available nodes are now all the successor nodes sorted by score
            available_nodes = sorted(successor_nodes, key=lambda x: x[1])

            # Choose only the k best successor states
            if k < len(available_nodes):
                available_nodes = available_nodes[:k]
            if success == True:
            	break  

    def success(self, node_dict, num_nodes_generated, print_solution=True):
        # When the solution is found, prints the solution path and the length of the solution path
        if len(node_dict) >= 1:

            # Find the final node
            for node_num, node in node_dict.items():
                if node["state"] == self.goalState:
                    final_node = node_dict[node_num]
                    break

            # Generate the solution path from the final node to the start node
            solution_path = self.generateSolutionPath(final_node, node_dict, path=[([[0, 1, 2], [3, 4, 5], [6, 7, 8]], "goal")])
            solution_length = len(solution_path) - 2

        else:
            solution_path = []
            solution_length = 0
        
        self.solution_path = solution_path 

        if print_solution:
            # Display the length of solution and solution path
            print("Solution found!")
            print("Solution Length: ", solution_length)

            # The solution path goes from final to start node. To display sequence of actions, reverse the solution path
            print("Solution Path", list(map(lambda x: x[1], solution_path[::-1])))
            print("Total nodes generated:", num_nodes_generated)
        
    def generateSolutionPath(self, node, node_dict, path):
        # Return the solution path for display from final (goal) state to starting state
        # If the node is the root, return the path
        if node["parent"] == "root":
            # If root is found, add the node and then return
            path.append((node["state"], node["action"]))
            return path

        else:
            # If the node is not the root, add the state and action to the solution path
            state = node["state"]
            parent_state = node["parent"]
            action = node["action"]
            path.append((state, action))

            # Find the parent of the node and recurse
            for node_num, expanded_node in node_dict.items():
                if expanded_node["state"] == parent_state:
                    return self.generateSolutionPath(expanded_node, node_dict, path)

In [291]:
puzzle = Puzzle()
puzzle.maxNodes(5000)
puzzle.printState()

012 345 678


In [292]:
puzzle.setState("035 846 271")
puzzle.printState()

035 846 271


In [293]:
puzzle.randomizeState(500)
puzzle.printState()

431 672 850


In [294]:
puzzle.beam()

Solution found!
Solution Length:  12
Solution Path ['start', 'left', 'left', 'up', 'up', 'right', 'right', 'down', 'down', 'left', 'up', 'left', 'up', 'goal']
Total nodes generated: 169


In [295]:
puzzle.printState()

431 672 850


In [297]:
puzzle.printState()

431 672 850


In [304]:
puzzle.randomizeState(500)
puzzle.printState()

021 673 548


In [305]:
puzzle.A_star("h1")

No Solution Found in first 10000 nodes generated


In [306]:
puzzle.A_star("h2")

Solution found!
Solution Length:  22
Solution Path ['start', 'down', 'right', 'right', 'up', 'left', 'down', 'right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'down', 'right', 'up', 'left', 'down', 'left', 'up', 'up', 'goal']
Total nodes generated: 3023


In [307]:
puzzle.beam()

Solution found!
Solution Length:  40
Solution Path ['start', 'down', 'down', 'right', 'up', 'left', 'up', 'right', 'right', 'down', 'left', 'down', 'left', 'up', 'right', 'up', 'left', 'down', 'right', 'down', 'right', 'up', 'up', 'left', 'left', 'down', 'right', 'right', 'down', 'left', 'left', 'up', 'right', 'up', 'left', 'down', 'down', 'right', 'up', 'left', 'up', 'goal']
Total nodes generated: 544
